<a href="https://colab.research.google.com/github/JammieBurger/imlo-project-actual/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialising Training and Testing Data.

In [ ]:
import torch
from torch import nn
from torch import _dynamo
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
training_data = datasets.Flowers102(
    root="data",
    split='train',
    transform=transform,
    download=True)
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform)

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data,batch_size)
test_dataloader = DataLoader(test_data,batch_size)
#for X, y in test_dataloader:
    #print(f"Shape of X [N, C, H, W]: {X.shape}")
    #print(f"Shape of y: {y.shape} {y.dtype}")
    #break
device = ("cuda"if torch.cuda.is_available()
                else "mps"
                if torch.backends.mps.is_available()
                else "cpu")
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 32, (3,3), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Sigmoid(),
            nn.Conv2d(32, 64, (3,3), (1,1), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            nn.BatchNorm2d(64),

            nn.Conv2d(64, 128, (3,3), (1,1), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Sigmoid(),
            nn.Conv2d(128, 128, (3,3), (1,1), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, (3,3), (1,1), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Sigmoid(),
            nn.Conv2d(256, 256, (3,3), (1,1), (1,1)),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.Flatten(),
            nn.Linear(2304, 512),
            nn.ReLU(),
            nn.Linear(512, 102))
            #nn.ReLU(),
            #nn.Linear(512, 102))



    def forward(self, x):
        x = self.layers(x)

        return x

Using cuda device


In [ ]:
model = NeuralNetwork().to(device)
#print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model = NeuralNetwork()
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "C:\Users\adman\OneDrive\Documents\Jamie\UniWork\IMLOmodel.pth")
print("Saved PyTorch Model State to model.pth")
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Epoch 1
-------------------------------
loss: 4.756352  [   64/ 1020]
Test Error: 
 Accuracy: 1.3%, Avg loss: 4.634808 

Epoch 2
-------------------------------
loss: 4.205650  [   64/ 1020]
Test Error: 
 Accuracy: 1.9%, Avg loss: 4.619455 

Epoch 3
-------------------------------
loss: 4.106983  [   64/ 1020]
Test Error: 
 Accuracy: 4.2%, Avg loss: 4.521866 

Epoch 4
-------------------------------
loss: 3.851990  [   64/ 1020]
Test Error: 
 Accuracy: 8.0%, Avg loss: 4.347646 

Epoch 5
-------------------------------
loss: 3.571502  [   64/ 1020]
Test Error: 
 Accuracy: 12.2%, Avg loss: 4.160403 

Epoch 6
-------------------------------
loss: 3.311050  [   64/ 1020]
Test Error: 
 Accuracy: 13.8%, Avg loss: 3.998466 

Epoch 7
-------------------------------
loss: 2.983920  [   64/ 1020]
Test Error: 
 Accuracy: 15.9%, Avg loss: 3.856353 

Epoch 8
-------------------------------
loss: 2.703921  [   64/ 1020]
Test Error: 
 Accuracy: 17.5%, Avg loss: 3.734203 

Epoch 9
--------------------